In [ ]:
%matplotlib inline 
import geopandas as gpd
import pandas as pd
from arcgis import *
import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
from IPython.display import display
import numpy as np


# add packages as needed

import os
current_dir = os.getcwd()
print(current_dir)

In [ ]:
gis = GIS(username='hnavnitl_UCSDOnline')

In [ ]:
# 1. Load the data (using concent manager, and also adding downloaded datasets), retain only fields we need
# You code goes here



In [ ]:
collisions = pd.read_csv('Collisions.csv')
collisions = collisions[["CASE_ID","COLLISION_SEVERITY","NUMBER_KILLED","NUMBER_INJURED",
                         "PARTY_COUNT","PRIMARY_COLL_FACTOR","PEDESTRIAN_ACCIDENT",
                         "BICYCLE_ACCIDENT","MOTORCYCLE_ACCIDENT","TRUCK_ACCIDENT",
                         "ALCOHOL_INVOLVED","LATITUDE","LONGITUDE","COUNTY","CITY",
                         "POINT_X","POINT_Y"]]
parties = pd.read_csv('Parties.csv')
parties = parties[["CASE_ID","PARTY_NUMBER","PARTY_TYPE", 'PARTY_SOBRIETY',"AT_FAULT",
                   "DIR_OF_TRAVEL"]]
victims = pd.read_csv('Victims.csv')
parties['PARTY_TYPE'] = parties['PARTY_TYPE'].replace('-', '')
parties['PARTY_TYPE'] = pd.to_numeric(parties['PARTY_TYPE'])

In [ ]:
party = parties[['PARTY_TYPE', 'CASE_ID']].set_index('CASE_ID')
party = pd.get_dummies(party['PARTY_TYPE'])
party.columns = ['DRIVER', 'PED', 'PARKED', "BIKE", 'OTHER']
party = party.groupby(['CASE_ID']).sum().reset_index()
party['NUM_PARTIES'] = party['DRIVER'] + party['PED'] + party['PARKED'] + party['BIKE'] + party['OTHER']
parties = pd.merge(parties, party, how='inner', on='CASE_ID')

In [ ]:
# 2. Joining party (and  victims, if going for extra credit) to collisions. 
# You code goes here

In [ ]:
merged = pd.merge(parties, collisions, on=['CASE_ID'])

In [ ]:
# 3. Clean spatial data as needed and create a point layer of collisions
# You code goes here

In [ ]:
merged['LATITUDE'] = merged['LATITUDE'].fillna(merged['POINT_Y'])
merged['LONGITUDE'] = merged['LONGITUDE'].fillna(merged['POINT_X'])
unique = merged.drop_duplicates(subset = 'CASE_ID')
unique = merged[['CASE_ID', 'LATITUDE', 'LONGITUDE', 'ALCOHOL_INVOLVED']]
sdf = pd.DataFrame.spatial.from_xy(unique, x_column = 'LONGITUDE', y_column = 'LATITUDE')
sdf = sdf.replace(np.nan, '', regex=True)
point_layer = sdf.spatial.to_featurelayer(title='Unique', gis=gis, tags='bike')
point_layer.share(org=True)

In [ ]:
# 4. Define what you mean to be on a bike route, 
# and figure out, for each collision, whether it happened on a bike route or not. 
# Summarize collisions of different types by zip codes.
# You code goes here



In [ ]:
san_bike_routes = gis.content.search('title: MP3_BIKE_ROUTES owner: boh016_UCSDOnline',
                                         'Feature layer',
                                          outside_org=False)[0]

san_bike_routes

bike_routes = san_bike_routes.layers[0]
bike_routes



In [ ]:
for lyr in san_bike_routes.layers:
    print(lyr.properties.name)

In [ ]:
bike_street_filtered_map = gis.map('San Diego')
bike_street_filtered_map

In [ ]:
# bike_street_filtered_map.add_layer({"type":"FeatureLayer", 
#               "url":bike_routes.url,
#               "definition_expression" : "SEGMENT_TY = 8", # code for trails
#              })
#bike_street_filtered_map
bike_street_filtered_map.add_layer(bike_routes)

In [ ]:
item = gis.content.get("08c0a2e88b55479389e25ca1d0fc7326")
item

In [ ]:
bike_street_filtered_map.add_layer(item)

In [ ]:
from arcgis.features.use_proximity import create_buffers

In [ ]:
buffer_street = create_buffers(item,
                               distances=[0.01],
                               ring_type='Rings',
                               units='Miles')

In [ ]:
bike_street_filtered_map.add_layer(buffer_street)

In [ ]:
routes = pd.DataFrame.spatial.from_layer(bike_routes)
#print(tract_routes.head())

#bike_points = item.layers[0]
#print(tract_points.head())

#joint_points = tract_routes.join(tract_points, on = 'SHAPE', how='left')
#joint_points

In [ ]:
# 5. Generate maps
# You code goes here

In [ ]:
# Map of whether alcohol involved
alc_map = gis.map('San Diego', zoomlevel = 10)
alc_map.add_layer(bike_routes)
alc_renderer = {"renderer": "autocast", "type": "uniqueValue", "field1":"ALCOHOL_IN"}

alc_renderer["uniqueValueInfos"] = [{  "value": "Y",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/PeoplePlaces/Bar.png","contentType":"image/png","width":18,"height":18
                                        },
                                        "label": "Alcohol Involved"
                                    }]
alc_map.add_layer(collisions, {'type': 'FeatureLayer',
                              'renderer': alc_renderer,
                              'field_name':'ALCOHOL_IN'})
alc_map

In [ ]:
impaired = parties.loc[parties['PARTY_TYPE'] == 4]
impaired = impaired.loc[impaired['AT_FAULT'] == 'Y']
sdf = pd.DataFrame.spatial.from_xy(merged, x_column = 'LONGITUDE', y_column = 'LATITUDE')
sdf = sdf.replace(np.nan, '', regex=True)
point_layer = sdf.spatial.to_featurelayer(title='Drunk Cyclist', gis=gis, tags='bike')
point_layer.share(org=True)

In [ ]:
# Map of whether bicyclist was at fault because of impairment
drunk_bike = gis.content.get("01b2bb984293475badec273228ca70ae")
drunk_bike_map = gis.map('San Diego', zoomlevel = 10)
drunk_bike_map.add_layer(bike_routes)
drunk_bike_renderer = {"renderer": "autocast", "type": "uniqueValue", "field1":"PARTY_SOBR"}
drunk_bike_renderer["uniqueValueInfos"] = [{  "value": "A",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/RedSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Had Not Been Drinking"
                                    },
                                          {  "value": "B",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/OrangeSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Had Been Drinking, Under Influence"
                                    },
                                          {  "value": "C",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/RedSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Had Been Drinking, Not Under Influence"
                                    },
                                          {  "value": "D",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/BlueSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Had Been Drinking, Impairment Unknown"
                                    },
                                          {  "value": "G",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/BlueSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Impairment Unknown"
                                    }]
drunk_bike_map.add_layer(drunk_bike, {'type': 'FeatureLayer',
                              'renderer': drunk_bike_renderer,
                              'field_name':'PARTY_SOBR'})
drunk_bike_map

In [ ]:
types = merged.loc[merged['NUM_PARTIES'] < 4]
types['CRASH_TYPE'] = np.nan

types.loc[(types['DRIVER'] == 1) & (types['NUM_PARTIES'] == 2), 'CRASH_TYPE'] = 'CAR IN 2 PARTY CRASH'
types.loc[(types['PED'] == 1) & (types['NUM_PARTIES'] == 2), 'CRASH_TYPE'] = 'PED IN 2 PARTY CRASH'
types.loc[(types['PARKED'] == 1) & (types['NUM_PARTIES'] == 2), 'CRASH_TYPE'] = 'PARKED CAR IN 2 PARTY CRASH'
types.loc[(types['BIKE'] == 1) & (types['NUM_PARTIES'] == 2), 'CRASH_TYPE'] = 'BIKE IN 2 PARTY CRASH'

types.loc[(types['PED'] > 1) & (types['NUM_PARTIES'] == 3), 'CRASH_TYPE'] = '2 PED IN 3 PARTY CRASH'
types.loc[(types['DRIVER'] > 1) & (types['NUM_PARTIES'] == 3), 'CRASH_TYPE'] = '2 CAR IN 3 PARTY CRASH'
types.loc[(types['PARKED'] > 1) & (types['NUM_PARTIES'] == 3), 'CRASH_TYPE'] = '2 PARKED IN 3 PARTY CRASH'
types.loc[(types['BIKE'] > 1) & (types['NUM_PARTIES'] == 3), 'CRASH_TYPE'] = '2 BIKE IN 3 PARTY CRASH'

types.loc[types['NUM_PARTIES'] == 1, 'CRASH_TYPE'] = 'NO OTHER PARTIES'
types['CRASH_TYPE'] = types['CRASH_TYPE'].fillna('Varied Vehicles involved')
types = types[['CRASH_TYPE', 'LONGITUDE', 'LATITUDE']]

sdf = pd.DataFrame.spatial.from_xy(types, x_column = 'LONGITUDE', y_column = 'LATITUDE')
sdf = sdf.replace(np.nan, '', regex=True)
point_layer = sdf.spatial.to_featurelayer(title='Crash Types', gis=gis, tags='bike')
point_layer.share(org=True)

In [ ]:
# Map of types of crashes
crash_types = gis.content.get("1ea00e20ec3347c4acf4c819a6b6bda5")
crash_types_map = gis.map('San Diego', zoomlevel = 10)
crash_types_map.add_layer(bike_routes)
crash_types_renderer = {"renderer": "autocast", "type": "uniqueValue", "field1":"CRASH_TYPE"}
crash_types_renderer["uniqueValueInfos"] = [{  "value": "CAR IN 2 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/OrangeSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "CAR IN 2 PARTY CRASH"
                                    },
                                          {  "value": "PED IN 2 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/PurpleSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "PED IN 2 PARTY CRASH"
                                    },
                                          {  "value": "PARKED CAR IN 2 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/GreenSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "PARKED CAR IN 2 PARTY CRASH"
                                    },
                                          {  "value": "BIKE IN 2 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/YellowSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "BIKE IN 2 PARTY CRASH"
                                    },
                                          {  "value": "2 PED IN 3 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/PurpleSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "2 PED IN 3 PARTY CRASH"
                                    },
                                        {  "value": "2 CAR IN 3 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/OrangeSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "2 CAR IN 3 PARTY CRASH"
                                    },
                                          {  "value": "2 PARKED IN 3 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/GreenSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "2 PARKED IN 3 PARTY CRASH"
                                    },
                                          {  "value": "2 BIKE IN 3 PARTY CRASH",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/YellowSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "2 BIKE IN 3 PARTY CRASH"
                                    },
                                          {  "value": "NO OTHER PARTIES",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/BlueSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "NO OTHER PARTIES"
                                    },
                                        {  "value": "Varied Vehicles involved",
                                        "symbol": {"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/RedSphere.png","contentType":"image/png","width":14,"height":14
                                        },
                                        "label": "Varied Vehicles involved"
                                    }]
crash_types_map.add_layer(crash_types, {'type': 'FeatureLayer',
                              'renderer': crash_types_renderer,
                              'field_name':'CRASH_TYPE'})
crash_types_map

In [ ]:
zip_codes = gis.content.get('81cf0db99f754a0ebc6f5ddaefd7bcad')

drunk = merged.loc[parties['PARTY_TYPE'] == 4]
drunk = drunk[['LONGITUDE', 'LATITUDE']]

sdf = pd.DataFrame.spatial.from_xy(drunk, x_column = 'LONGITUDE', y_column = 'LATITUDE')
sdf = sdf.replace(np.nan, '', regex=True)
point_layer = sdf.spatial.to_featurelayer(title='Alcohol', gis=gis, tags='bike')
point_layer.share(org=True)

In [ ]:
# Map of drunk collisions separated by zip code
drunk_col = gis.content.get('4f65952e4c554590add8dab7de9457d7')
drunk_map = gis.map('San Diego', zoomlevel = 10)
drunk_map.add_layer(zip_codes)
drunk_map.add_layer(drunk_col)

drunk_map

In [ ]:
# 6. Analyze the data and write responses to the above questions
# You code goes here

Through our data analysis we found that it is less likely to be injured in a bike-related accident if you are on a bike path. This is likely due to the fact that cars and other people on the road are aware of where a biker is going to be. We also examined types of parties involved in crashes and determined the various categories to be either a 1 party accident, a 2 party accident, or a 3 party accidents with varied numbers of each party type. We decided to exclude instances of collisions where there were more than 3 parties involved such as cases where there were 4, 5, or even 12 parties involved since each of these collisions only occurred once. In doing this analysis we found that the in bike accidents, the other party is usually a car, regardless of whether two or three parties were involved. We also found that in accidents where the cylist was the party at fault, very often either they had not been drinking or they had been drinking but were not under the influence. This was suprising to us given that we had originally thought many accidents must have been caused by intoxication. However, there was still a good number of accidents that did have alcohol involved in general.

URLS Created
https://ucsdonline.maps.arcgis.com/home/item.html?id=08c0a2e88b55479389e25ca1d0fc7326 https://ucsdonline.maps.arcgis.com/home/item.html?id=6f2e0b3b36fa49689ceb6a1337c78a56 https://ucsdonline.maps.arcgis.com/home/item.html?id=01b2bb984293475badec273228ca70ae https://ucsdonline.maps.arcgis.com/home/item.html?id=1ea00e20ec3347c4acf4c819a6b6bda5 https://ucsdonline.maps.arcgis.com/home/item.html?id=4f65952e4c554590add8dab7de9457d7